In [35]:
from __future__ import print_function
import time

############################################
# Config
##########################################

"""
Setting debug to true will display more informations
about the lattice, the bounds, the vectors...
Setting debug_ext to true will display even more informations
about the lattice, the bounds, the vectors...
"""
debug = False
debug_ext = False

############################################
# Functions
##########################################

# display matrix picture with 0 and X
def matrix_overview(BB, bound):
    for ii in range(BB.dimensions()[0]):
        a = ('%02d ' % ii)
        for jj in range(BB.dimensions()[1]):
            a += '0' if BB[ii,jj] == 0 else '*'
            if BB.dimensions()[0] < 60:
                a += ' '
        # if BB[ii, ii] >= bound:
        #     a += '~'
        print(a)

In [36]:
def kunihiro(pol, modulus, mm, tt, rr, XX, YY):

    PR.<y, u, x> = PolynomialRing(ZZ, 3, order='lex')
    Q = PR.quotient(x*y^rr - 1 - u) # u = xy^r - 1
    polZ = Q(pol).lift()

    UU = XX*YY^rr + 1
    tau = tt/mm

    # xy-shifts
    gg = []
    for kk in range(mm+1):
        for jj in range(rr+ceil(tau*kk)):
            if jj < rr:
                for ii in range(kk+1):
                    xshift = x^(kk-ii)*y^jj * modulus^(mm - ii) * polZ(y, u, x)^ii  
                    
                    xshift = Q(xshift).lift()
                    if debug_ext:
                        print(jj, kk, ii, xshift.monomials())
                    gg.append(xshift)
            else:
                xshift = y^jj * modulus^(mm - kk) * polZ(y, u, x)^kk
            
                xshift = Q(xshift).lift()
                if debug_ext:
                    print(jj, kk, ii, xshift.monomials())
                gg.append(xshift)

    # xy-shifts list of monomials
    monomials = []
    for polynomial in gg:
        for monomial in polynomial.monomials():
            if monomial not in monomials:
                monomials.append(monomial)
    monomials.sort()
    
    if debug_ext:
        print(monomials)

    # construct lattice B
    nn = len(monomials)
    BB = Matrix(ZZ, nn)
    for ii in range(nn):
        for jj in range(nn):
            if monomials[jj] in gg[ii].monomials():
                BB[ii, jj] = gg[ii].monomial_coefficient(monomials[jj]) * monomials[jj](YY, UU, XX)

    # display the lattice basis
    if debug:
        matrix_overview(BB, modulus^mm)

    # LLL
    if debug:
        print("optimizing basis of the lattice via LLL, this can take a long time")

    BB = BB.LLL()

    if debug:
        print("LLL is done!")

    # transform vector i & j -> polynomials 1 & 2
    if debug:
        print("looking for independent vectors in the lattice")
    found_polynomials = False
    
    for pol1_idx in range(nn - 1):
        for pol2_idx in range(pol1_idx + 1, nn):
            # for i and j, create the two polynomials
            PR.<z,w> = PolynomialRing(ZZ)
            pol1 = pol2 = 0
            for jj in range(nn):
                pol1 += monomials[jj](z,w*z^rr - 1,w) * BB[pol1_idx, jj] / monomials[jj](YY, UU, XX)
                pol2 += monomials[jj](z,w*z^rr - 1,w) * BB[pol2_idx, jj] / monomials[jj](YY, UU, XX)

            # resultant
            PR.<q> = PolynomialRing(ZZ)
            res = pol1.resultant(pol2)

            # are these good polynomials?
            if res.is_zero() or res.monomials() == [1]:
                continue
            else:
                print("found them, using vectors", pol1_idx, "and", pol2_idx)
                found_polynomials = True
                break
        if found_polynomials:
            break

    if not found_polynomials:
        print("no independant vectors could be found. This should very rarely happen...")
        return 0, 0

    res = res(q, q)

    # solutions
    soly = res.roots()

    if len(soly) == 0:
        print("Your prediction (delta) is too small")
        return 0, 0

    soly = soly[0][0]
    ss = pol1(q, soly)
    solx = ss.roots()[0][0]

    return solx, soly

In [38]:
def example():
    ############################################
    # How To Use This Script
    ##########################################

    # the modulus
    N = 3014972633503040336590226508316351022768913323933
    # the public exponent
    e = 653321922931937515584165279485561643717311696551550689661966133765127824043894656155756279180095177299327928182942709277283882982169138979615253
    r = 3

    # the hypothesis on the private exponent
    beta = .6
    gamma = .5
    
    #
    # Lattice (tweak those values)
    #
    # you should tweak this (after a first run), (e.g. increment it until a solution is found)
    m = t = 5 # size of the lattice (bigger the better/slower)

    # you need to be a lattice master to tweak these
    X = floor(N^beta)
    Y = floor(N^gamma)
    Z = X*Y^r + 1

    # Problem put in equation
    P.<x,y> = PolynomialRing(ZZ)
    h = N^3 + 3*N*y - y^3 + 1
    h = -h
    pol = x*h-1

    #
    # find the solutions!
    #

    # bounds
    if debug:
        print("=== checking values ===")
        print("* e: ", e)
        print("* size of e:", int(log(e)/log(2)))
        print("* N: ", N)
        print("* size of N:", int(log(N)/log(2)))
        alpha = float(log(e)/log(N))
        print("* size of alpha:", alpha)
        print("* m:", m, ", t:", t)
        print("* X: ", X)
        print("* Y: ", Y)
        print("* Z: ", Z)
        print("* size of Z:", float(log(Z)/log(N)))

    # kunihiro
    print("=== running algorithm ===")
    start_time = time.time()

    solx, soly = kunihiro(pol, e, m, t, r, X, Y)

    # found a solution?
    if solx > 0:
        print("=== solution found ===")
        # if False:
        print("x:", solx)
        print("y:", soly)

        PV.<t> = PolynomialRing(ZZ)
        viete = t^2 - solx * t + N
        sol_viete = viete.roots()
        
        print("=== primes found ===")

        p = sol_viete[0][0]
        q = sol_viete[1][0]

        print("p: ", p)
        print("q: ", q)

        print("Correct primes?", (N == p*q))
        
    else:
        print("=== no solution was found ===")

    print(("=== %s seconds ===" % (time.time() - start_time)))

if __name__ == "__main__":
    example()

=== running algorithm ===
found them, using vectors 0 and 9
=== solution found ===
x: 3542083907659073025514626
y: 2916400291365712080420733503
=== primes found ===
p:  2119778199036859068707819
q:  1422305708622213956806807
Correct primes? True
=== 5.897770166397095 seconds ===
